# Frequentist Inference Case Study - Part B

## Learning objectives

Welcome to Part B of the Frequentist inference case study! The purpose of this case study is to help you apply the concepts associated with Frequentist inference in Python. In particular, you'll practice writing Python code to apply the following statistical concepts: 
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, including its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate a confidence interval

In the previous notebook, we used only data from a known normal distribution. **You'll now tackle real data, rather than simulated data, and answer some relevant real-world business problems using the data.**

## Hospital medical charges

Imagine that a hospital has hired you as their data scientist. An administrator is working on the hospital's business operations plan and needs you to help them answer some business questions. 

In this assignment notebook, you're going to use frequentist statistical inference on a data sample to answer the questions:
* has the hospital's revenue stream fallen below a key threshold?
* are patients with insurance really charged different amounts than those without?

Answering that last question with a frequentist approach makes some assumptions, and requires some knowledge, about the two groups.

We are going to use some data on medical charges obtained from [Kaggle](https://www.kaggle.com/easonlai/sample-insurance-claim-prediction-dataset). 

For the purposes of this exercise, assume the observations are the result of random sampling from our single hospital. Recall that in the previous assignment, we introduced the Central Limit Theorem (CLT), and its consequence that the distributions of sample statistics approach a normal distribution as $n$ increases. The amazing thing about this is that it applies to the sampling distributions of statistics that have been calculated from even highly non-normal distributions of data! Recall, also, that hypothesis testing is very much based on making inferences about such sample statistics. You're going to rely heavily on the CLT to apply frequentist (parametric) tests to answer the questions in this notebook.

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy import stats
from scipy.stats import norm
from numpy.random import seed

plt.style.use(['dark_background'])
medical = pd.read_csv('insurance2.csv')

In [ ]:
medical.shape

In [ ]:
medical.head()

__Q1:__ Plot the histogram of charges and calculate the mean and standard deviation. Comment on the appropriateness of these statistics for the data.

__A:__

In [ ]:
charges = medical['charges']

# 30 bins were used as len(charges) ^ 0.5 is around 30
_ = plt.hist(charges, bins=30)
_ = plt.xlabel('Charges')
_ = plt.title('Medical Charges')
plt.show()

# std with bessel adjustment is used as we are using a sample and not the whole pop of observations from hospital
std = np.std(charges, ddof=1)

# mean
mean = np.mean(charges)

In [ ]:
print('std:', std, 'mean:', mean)

__Q2:__ The administrator is concerned that the actual average charge has fallen below 12,000, threatening the hospital's operational model. On the assumption that these data represent a random sample of charges, how would you justify that these data allow you to answer that question? And what would be the most appropriate frequentist test, of the ones discussed so far, to apply?

__A:__ A t-test would be most appropriate because we do not know the population standard deviation. Also we can justify that these random sampling
would be representative of our target population, according to a property of statistics that state that we only need a well-randomized
sample of the population to get taste of the population distribution; rather than testing with the whole target population.


__Q3:__ Given the nature of the administrator's concern, what is the appropriate confidence interval in this case? A ***one-sided*** or ***two-sided*** interval? (Refresh your understanding of this concept on p. 399 of the *AoS*). Calculate the critical value and the relevant 95% confidence interval for the mean, and comment on whether the administrator should be concerned.

__A:__ A one tailed test is used because we are only interested in a specific direction which are charges below 12,000; and from results
we can be 95% confident that charge is at approximately $12,725.49 and above; which means the administration should not be concerned.

In [ ]:
critical_val_t = t.ppf(0.95, len(charges) - 1)


In [ ]:
standard_error = stats.sem(charges)

In [ ]:
CI_95 = mean - critical_val_t * standard_error
CI_95


The administrator then wants to know whether people with insurance really are charged a different amount to those without.

__Q4:__ State the null and alternative hypothesis here. Use the _t_-test for the difference between means, where the pooled standard deviation of the two groups is given by:

\begin{equation}
s_p = \sqrt{\frac{(n_0 - 1)s^2_0 + (n_1 - 1)s^2_1}{n_0 + n_1 - 2}}
\end{equation}

and the *t*-test statistic is then given by:

\begin{equation}
t = \frac{\bar{x}_0 - \bar{x}_1}{s_p \sqrt{1/n_0 + 1/n_1}}.
\end{equation}

(If you need some reminding of the general definition of ***t-statistic***, check out the definition on p. 404 of *AoS*). 

What assumption about the variances of the two groups are we making here?

__A:__
* H0 =  On average, people that have insurance are charged the same amount as people without it.
* Ha = On average, the charges between the two groups are significantly different.
* Also the test would need to assume that the variance between the two groups are same

__Q5:__ Perform this hypothesis test both manually, using the above formulae, and then using the appropriate function from [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests) (hint, you're looking for a function to perform a _t_-test on two independent samples). For the manual approach, calculate the value of the test statistic and then its probability (the p-value). Verify you get the same results from both.

__A:__ 

In [53]:
medical.head()
insured = medical.loc[medical.insuranceclaim == 1].charges
non_insured = medical.loc[medical.insuranceclaim == 0].charges

mean_insured = insured.mean()
mean_non_insured = non_insured.mean()

std_insured = np.std(insured, ddof=1)
std_non_insured = np.std(non_insured, ddof=1)

x0, x1 = mean_insured, mean_non_insured
s0, s1 = std_insured, std_non_insured
n0, n1 = len(insured), len(non_insured)

# sp is ppoled standard dev
manual_sp = np.sqrt(((n0 - 1) * s0**2 + (n1 - 1) * s1**2) / (n0 + n1 - 2))

manual_t = (x0 - x1) / (manual_sp * np.sqrt(1 / n0 + 1 / n1))
manual_p = 2 * t.sf(np.abs(manual_t), n0 + n1 - 2)

In [55]:
print('manual values for t and p are ', 'test statistic:', manual_t, 'p value:', manual_p)


manual values for t and p are  test statistic: 11.89329903087671 p value: 4.461230231620972e-31


In [56]:
func_t, func_p = stats.ttest_ind(insured, non_insured, equal_var=True)

In [58]:
print('functional values for t and p are ', 'test statistic:', func_t, 'p value:', func_p)


functional values for t and p are  test statistic: 11.893299030876712 p value: 4.461230231620717e-31


Congratulations! Hopefully you got the exact same numerical results. This shows that you correctly calculated the numbers by hand. Secondly, you used the correct function and saw that it's much easier to use. All you need to do is pass your data to it.

__Q6:__ Conceptual question: look through the documentation for statistical test functions in scipy.stats. You'll see the above _t_-test for a sample, but can you see an equivalent one for performing a *z*-test from a sample? Comment on your answer.

__A:__ Currently there is none

## Learning outcomes

Having completed this project notebook, you now have good hands-on experience:
* using the central limit theorem to help you apply frequentist techniques to answer questions that pertain to very non-normally distributed data from the real world
* performing inference using such data to answer business questions
* forming a hypothesis and framing the null and alternative hypotheses
* testing this using a _t_-test